# Commodity Food Pricing: Model Experiments

**By:** `MSOE AI-Club "Nourish" Student Research Team`<br/>
**Primary Notebook Developer:** Ben Paulson, ____

**Notebook Purpose:** To explore the use of machine learning to predict the price of commodities in the food industry. By utilizing the clean datasets loaded from `data_analysis.ipynb` into the directory `model_data`, we can begin to explore the use of machine learning to predict the price of commodities in the food industry. This notebook will be divided into sections of varying complexity, each of which will explore a different model type and its performance on the data. The models will be evaluated based on their ability to predict the price of a commodity in the future, given a set of features. The features will be selected based on their correlation to the price of the commodity, as determined in `data_analysis.ipynb`. There is also significant documentation with each section for both code/purpose sanity, sake of future reproducibility, and to help other members of the `MSOE AI-Club "Nourish" Student Research Team` understand the code and its purpose.
* **Part 1: Building Complex ML Models**
    * **Model 1:** Simple, 1-layer Transformer
    * **Model 2:** ...
* **Part 2: Experimenting with ML Models**
    * **Experiment 1:** Using Only Previous Price Data With a Simple Transformer
    * **Experiment 2:** ...

**Research Context:** This research is being conducted as part of the MSOE AI-Club's "Nourish" project, which aims to use machine learning to predict future food prices in order to help farmers in developing countries make better decisions about food storage and crop selection, achieved through the accurate warning administration and prediction of food commodity pricing data by country and food market. This project is pending as part of a relationship with the United Nation's Food and Agriculture Organization (FAO). [FAO Official Statement on the Importance of Research Like This](https://youtu.be/sZx3hhnEHiI)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf

In [6]:
# Grabbing the data we hope to use throughout this experiment

model_data = pd.read_csv('model_data/argentina_wheat_model_data.csv')
model_data

,Unnamed: 0,Date,Price,Proteus2,Food Price Index,Cereals Price Index,Wheat Futures,Harvest,Sentiment
0,0,2016-12-01,0.351207,0.252506,0.647473,0.540047,405.90625,2679728.0,1.0
1,1,2016-12-01,0.351207,0.252506,0.647473,0.540047,405.90625,23367493.0,1.0
2,2,2016-12-01,0.351207,0.252506,0.647473,0.540047,405.90625,2157181.0,1.0
3,3,2016-11-01,0.368768,0.252506,0.651951,0.538869,408.40625,2679728.0,1.0
4,4,2016-11-01,0.368768,0.252506,0.651951,0.538869,408.40625,23367493.0,1.0
...,...,...,...,...,...,...,...,...,...
787,787,2002-11-01,0.284833,0.292393,0.476008,0.490577,428.25000,3653782.0,1.0
788,788,2002-11-01,0.284833,0.292393,0.476008,0.490577,428.25000,6109417.0,1.0
789,789,2002-11-01,0.284833,0.292393,0.476008,0.490577,428.25000,31861336.0,1.0
790,790,2002-11-01,0.284833,0.292393,0.476008,0.490577,428.25000,1994419.0,1.0


## Part 1: Building Complex ML Models

In [4]:
# Define positional encoding function (another portion for data preprocessing)
def positional_encoding(length, depth):
    """ 
    Create positional encoding
    args:
        length: length of the sequence
        depth: depth of the model
    """
    pos_enc = np.array([
        [pos / np.power(10000, 2 * (j // 2) / depth) for j in range(depth)]
        for pos in range(length)
    ])
    pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])
    pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])
    return tf.cast(pos_enc, dtype=tf.float32)

### Model 1: Simple, 1-Layer Transformer

In [5]:
HEAD_SIZE = 64
NUM_HEADS = 16
FF_DIM = 16

In [3]:
def transformer(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-Head Attention
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = tf.keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = tf.keras.layers.Dropout(dropout)(x)
    
    # Add & Norm
    res = x + inputs  # Skip connection
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(res)
    
    # Feed-Forward Network (Using Dense layers instead of Conv1D layers)
    x = tf.keras.layers.Dense(ff_dim, activation='relu')(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    
    # Final Skip Connection
    x = x + res  # Skip connection
    
    return x

In [ ]:
# Define the input layer
inputs = tf.keras.Input(shape=(sequence_length, d_model))

# Add positional encoding to the input
# The positional encoding is added to the input in order to give the model some information about the relative position of the words in the sequence
# Not including the positional encoding is basically the same as randomizing the order of the data
x = inputs + positional_encoding(sequence_length, d_model) 

# Transformer Encoder
x = transformer(x, head_size=head_size, num_heads=num_heads, ff_dim=ff_dim)

# Global Average Pooling layer
# The Global Average Pooling layer reduces the dimensionality of the data
x = tf.keras.layers.GlobalAveragePooling1D()(x)

# Output layer
outputs = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()